In [3]:
#TRABALHO DE CONCLUSÃO

import requests
import pprint
import urllib
import json
import time
import pandas as pd
import csv
import re
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#Constantes setadas para limpeza de texto dos resumos.
PATTERN_S = re.compile("\'s")                # encontra os 's do texto 
PATTERN_RN = re.compile("\\r\\n")            # encontra os \r e \n do texto
PATTERN_PUNC = re.compile(r"[^\w\s]")        # encontra tudo que não é 0-9 A-z e ' '
STOPWORDS = set(stopwords.words('english'))  # palavras comuns em inglês, como 'and, of, the, etc'

#Seta URL para busca API. query é usada como teste
url = 'https://api.elsevier.com/content/search/scopus'
query = 'TF-IDF AND collaborative AND recommendation AND filtering AND similarity AND cosine AND mining AND NOT java'

#Seta os parametros e headers para a busca API
parameters = {
    'query': urllib.parse.quote_plus(query),
    'start': '0'
}

headers = {
    'Accept': 'application/json',
    'X-ELS-APIKey': 'f00003d0569e6d735211b697539544ea'
}

#Método usado para limpar o texto dos resumos
def clean_text(text):
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

#Método usado para remover as palavras que precedem NOT da query na vetorização
def limpaNOTpalavra(query):
    arr = query.split()
    indices = []
    for w in range(len(arr)):
        if w > 0:
            if arr[w-1] == 'NOT':
                indices.insert(0, w)
    for z in range(len(indices)):
        arr.pop(indices[z])            
    return ' '.join(arr)

#Método usado para limpar os operadores da query antes de aplicar PLN
def limpaOperadores(query):
    query = limpaNOTpalavra(query)
    palavras = ['AND', 'OR', 'NOT']
    arr = query.split()
    indices = []
    for w in range(len(arr)):
        for y in range(len(palavras)):
            if palavras[y] == arr[w]:
                indices.insert(0, w)
    for z in range(len(indices)):
        arr.pop(indices[z])
    indices = []
    return clean_text(' '.join(arr))
    
#Transforma a string resumo em lista de palavras, removendo plurais e padronizando variações
def tokenizer(sentence, stopwords=STOPWORDS, lemmatize=True):
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    tokens = [w for w in tokens if w not in stopwords]
    return tokens

#Método para extrair os melhores índices após cálculo de semelhança
def extract_best_indices(m, top):
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] #organiza do índice de maior resultado ao menor
    temp = np.ones(len(cos_sim))
    temp = np.logical_or(cos_sim[index] != 0, temp) #elimina cosseno com distância 0
    best_index = index[temp][:top]  
    return best_index

#Método principal; aplica a query na matriz, calcula semelhança e extrai os melhores resultados
def get_recommendations_tfidf(sentence, tfidf_mat):
    # Incorpora a query na matriz de vetores
    tokens_query = [str(tok) for tok in tokenizer(sentence)]
    embed_query = vectorizer.transform(tokens_query)
    # Cria uma lista com similaridade entre a query e o dataset
    mat = cosine_similarity(embed_query, tfidf_mat)
    # Recebe os 5 índices de melhor similaridade de cosseno para cada token.
    best_index = extract_best_indices(mat, top=5)
    return best_index

#Busca as URLs e títulos dos artigos retornados // Salva títulos em csv e retorna lista de URLs
def buscaURL_Titulo(data):
    listaURLS = []
    with open('teste.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['tituloArtigos'])
        iter = int(data["search-results"]["opensearch:totalResults"])
        print("A API do Scopus encontrou " + str(iter) + " resultados.")
        if  iter > 25: #25 é o número de resultados que o jupyter mostra
            if iter > 60:
                iter = 60 #limitando a busca para no máximo 60 resultados visto que pode levar até 3min pra terminar a execução
            for y in range(iter):
                if ((y+1) % 26) == 0:
                    parameters['start'] = str(int(parameters['start']) + 25)
                    response = requests.get(url, params=parameters, headers=headers)
                    data = response.json()
                listaURLS.append(data['search-results']['entry'][y%25]['link'][2]['@href']) #URLs são usadas para obter os resumos
                writer.writerow([data['search-results']['entry'][y%25]['dc:title']])        #Títulos são armazenados em .csv                
        else:
            for y in range(iter):
                listaURLS.append(data['search-results']['entry'][y]['link'][2]['@href']) 
                writer.writerow([data['search-results']['entry'][y]['dc:title']]) 
    return listaURLS
        
#Busca o resumo para cada URL adquirida anteriormente // Retorna lista de resumos populada
def buscaResumos(listaURLS):
    #Inicializa a automação Selenium para busca de resumos
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome('chromedriver', options=options)
    listaResumos = []
    for y in range(len(listaURLS)):
        robo = driver.get(listaURLS[y])
        #Identifica se o elemento está presente na página
        try:
            element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="abstractSection"]/p'))
            )
        except:
            print("O resumo não foi encontrado na página.")
        time.sleep(1)
        robo = driver.find_element(By.XPATH, '//*[@id="abstractSection"]/p').text
        #Caso o elemento não tenha sido corretamente adquirido, uma nova tentativa é feita após 3 segundos.
        if robo == '':
            time.sleep(3)
            robo = driver.find_element(By.XPATH, '//*[@id="abstractSection"]/p').text
        listaResumos.append(clean_text(robo))
        print("\r", "Carregando os resumos da busca..." + str(y+1) + " de " + str(len(listaURLS)), end="")
    driver.quit()
    return listaResumos

#Executa a busca API e transforma em json
response = requests.get(url, params=parameters, headers=headers)
data = response.json()
resumos = buscaResumos(buscaURL_Titulo(data))

# #Lê o .csv e adiciona os resumos aos títulos
tabela = pd.read_csv('teste.csv')
tabela['resumos'] = resumos

#Aplica os stopwords na vetorização da matriz
token_stop = tokenizer(' '.join(STOPWORDS), lemmatize=False)
vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer) 

#Constrói a matriz vetorizada dos resumos, matriz de tamanho numero-de-palavras x numero-do-vocabulario
tfidf_mat = vectorizer.fit_transform(tabela['resumos'].values)

#Pega os melhores 5* artigos com base na query providenciada (numero de artigos retornados pode alterar)
best_index = get_recommendations_tfidf(limpaOperadores(query), tfidf_mat)

#Mostra o título dos melhores artigos
display(tabela['tituloArtigos'].iloc[best_index])



A API do Scopus encontrou 48 resultados.
 Carregando os resumos da busca...48 de 48

7     Comparison of two methods on vector space mode...
18    Improved Collaborative Filtering Recommendatio...
46    Adapting vector space model to ranking-based c...
44    A hybrid content-based filtering approach: Rec...
10    Cross-Content Recommendation between Movie and...
Name: tituloArtigos, dtype: object